# Beautiful Soup

In [1]:
# The code was copied from ....
[https://www.promptcloud.com/blog/how-to-scrape-yahoo-finance-data-using-python/]

import urllib.request
import urllib.parse
import urllib.error
from bs4 import BeautifulSoup
import ssl
import json
import ast
import os
from urllib.request import Request, urlopen

# For ignoring SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [2]:
# Input from the user
# url = input('Enter Yahoo Finance Company Url- ')
# url = input('https://finance.yahoo.com/quote/F?p=F&.tsrc=fin-srch')
url = 'https://finance.yahoo.com/quote/F'

# Making the website believe that you are accessing it using a Mozilla browser
req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
webpage = urlopen(req).read()
# Creating a BeautifulSoup object of the HTML page for easy extraction of data.

soup = BeautifulSoup(webpage, 'html.parser')
html = soup.prettify('utf-8')
company_json = {}
other_details = {}
for span in soup.findAll('span',
                         attrs={'class': 'Trsdu(0.3s) Trsdu(0.3s) Fw(b) Fz(36px) Mb(-4px) D(b)'
                         }):
    company_json['PRESENT_VALUE'] = span.text.strip()
# for div in soup.findAll('div', attrs={'class': 'D(ib) Va(t)'}):
#     for span in div.findAll('span', recursive=False):
#         company_json['PRESENT_GROWTH'] = span.text.strip()
# for td in soup.findAll('td', attrs={'data-test': 'PREV_CLOSE-value'}):
#     for span in td.findAll('span', recursive=False):
#         other_details['PREV_CLOSE'] = span.text.strip()
# for td in soup.findAll('td', attrs={'data-test': 'OPEN-value'}):
#     for span in td.findAll('span', recursive=False):
#         other_details['OPEN'] = span.text.strip()
# for td in soup.findAll('td', attrs={'data-test': 'BID-value'}):
#     for span in td.findAll('span', recursive=False):
#         other_details['BID'] = span.text.strip()
# for td in soup.findAll('td', attrs={'data-test': 'ASK-value'}):
#     for span in td.findAll('span', recursive=False):
#         other_details['ASK'] = span.text.strip()
# for td in soup.findAll('td', attrs={'data-test': 'DAYS_RANGE-value'}):
#     for span in td.findAll('span', recursive=False):
#         other_details['DAYS_RANGE'] = span.text.strip()
# for td in soup.findAll('td',
#                        attrs={'data-test': 'FIFTY_TWO_WK_RANGE-value'}):
#     for span in td.findAll('span', recursive=False):
#         other_details['FIFTY_TWO_WK_RANGE'] = span.text.strip()
# for td in soup.findAll('td', attrs={'data-test': 'TD_VOLUME-value'}):
#     for span in td.findAll('span', recursive=False):
#         other_details['TD_VOLUME'] = span.text.strip()
# for td in soup.findAll('td',
#                        attrs={'data-test': 'AVERAGE_VOLUME_3MONTH-value'
#                        }):
#     for span in td.findAll('span', recursive=False):
#         other_details['AVERAGE_VOLUME_3MONTH'] = span.text.strip()
# for td in soup.findAll('td', attrs={'data-test': 'MARKET_CAP-value'}):
#     for span in td.findAll('span', recursive=False):
#         other_details['MARKET_CAP'] = span.text.strip()
# for td in soup.findAll('td', attrs={'data-test': 'BETA_3Y-value'}):
#     for span in td.findAll('span', recursive=False):
#         other_details['BETA_3Y'] = span.text.strip()
# for td in soup.findAll('td', attrs={'data-test': 'PE_RATIO-value'}):
#     for span in td.findAll('span', recursive=False):
#         other_details['PE_RATIO'] = span.text.strip()
# for td in soup.findAll('td', attrs={'data-test': 'EPS_RATIO-value'}):
#     for span in td.findAll('span', recursive=False):
#         other_details['EPS_RATIO'] = span.text.strip()
# for td in soup.findAll('td', attrs={'data-test': 'EARNINGS_DATE-value'
#                        }):
#     other_details['EARNINGS_DATE'] = []
#     for span in td.findAll('span', recursive=False):
#         other_details['EARNINGS_DATE'].append(span.text.strip())
for td in soup.findAll('td',
                       attrs={'data-test': 'DIVIDEND_AND_YIELD-value'}):
    other_details['DIVIDEND_AND_YIELD'] = td.text.strip()
# for td in soup.findAll('td',
#                        attrs={'data-test': 'EX_DIVIDEND_DATE-value'}):
#     for span in td.findAll('span', recursive=False):
#         other_details['EX_DIVIDEND_DATE'] = span.text.strip()
# for td in soup.findAll('td',
#                        attrs={'data-test': 'ONE_YEAR_TARGET_PRICE-value'
#                        }):
#     for span in td.findAll('span', recursive=False):
#         other_details['ONE_YEAR_TARGET_PRICE'] = span.text.strip()
company_json['OTHER_DETAILS'] = other_details
with open('data.json', 'w') as outfile:
    json.dump(company_json, outfile, indent=4)
print(company_json)
with open('output_file.html', 'wb') as file:
    file.write(html)
print('----------Extraction of data is complete. Check json file.----------')

{'PRESENT_VALUE': '10.02', 'OTHER_DETAILS': {'DIVIDEND_AND_YIELD': '0.60 (6.30%)'}}
----------Extraction of data is complete. Check json file.----------


# Scrapy  

Still testing this... if look like Yahoo may have changed attributes of their web page as it pertains to API's and scraping.  Not sure what the problem is.

In [1]:
# Importing in each cell because of the kernel restarts.
import scrapy
from scrapy.crawler import CrawlerProcess


class YHSpider(scrapy.Spider):
    # Naming the spider is important if you are running more than one spider of
    # this class simultaneously.
    name = "YH"
    
    # URL(s) to start with.
    start_urls = [
        'https://finance.yahoo.com/quote/F/',
    ]

    # Use XPath to parse the response we get.
    def parse(self, response):
        
        # Iterate over every <article> element on the page.
        for stock in response.xpath('//td'):
            print(stock)
#             print(stock.extract())

            print(stock.xpath('[@class="Ta(end) Fw(600) Lh(14px)"]').extract())
            
            
            # Yield a dictionary with the values we want.
            yield { 
                # This is the code to choose what we want to extract
                # You can modify this with other Xpath expressions to extract other information from the site
#                 'test': stock.xpath("text()").extract_first(),
                'dividend': stock.xpath('data-test[@class="DIVIDEND_AND_YIELD-value"]/text()').extract_first()
#                 'date': article.xpath('header/section/span[@class="entry-date"]/text()').extract_first(),
#                 'text': article.xpath('section[@class="entry-content"]/p/text()').extract(),
#                 'tags': article.xpath('*/span[@class="tag-links"]/a/text()').extract()
            }
        
# test_inst = YHSpider(scrapy.Spider)
# YHSpider.parse(response)
        
# Tell the script how to run the crawler by passing in settings.
process = CrawlerProcess({
    'FEED_FORMAT': 'json',         # Store data in JSON format.
    'FEED_URI': 'firstpage.json',  # Name our storage file.
    'LOG_ENABLED': False           # Turn off logging for now.
})

# Start the crawler with our spider.
process.crawl(YHSpider)
process.start()
print('Success!')


<Selector xpath='//td' data='<td class="C(black) W(51%)" data-reactid'>
Success!


In [1]:
import pandas as pd

firstpage = pd.read_json('firstpage.json', orient='records', lines=True)
print(firstpage.shape)
print(firstpage.head())

ValueError: Unexpected character found when decoding 'false'